**Тестируем код Антона**

In [27]:
import yfinance as yf
import pandas as pd

In [61]:
taken_tickers = ["IMOEX.ME", "META", "TSLA", "AAPL"]

df_final = pd.DataFrame()

for one_ticker in taken_tickers:
    asset = yf.Ticker(one_ticker)
    dfhelp = asset.history(period = "1y").reset_index()[["Date", "Close"]]
    dfhelp = dfhelp.rename(columns = {"Close":one_ticker})
    dfhelp["Date"] = dfhelp.Date.dt.date
    if len(df_final) == 0:
        df_final = dfhelp
    else:
        df_final = df_final.merge(dfhelp, on = ["Date"])

In [62]:
df_final

,Date,IMOEX.ME,META,TSLA,AAPL
0,2022-11-28,2177.449951,108.779999,182.919998,143.418365
1,2022-11-29,2185.320068,109.459999,180.830002,140.385315
2,2022-11-30,2174.530029,118.099998,194.699997,147.207169
3,2022-12-01,2187.290039,120.440002,194.699997,147.485626
4,2022-12-02,2180.439941,123.489998,194.860001,146.988403
...,...,...,...,...,...
240,2023-11-17,3206.340088,335.040009,234.300003,189.690002
241,2023-11-20,3207.360107,339.970001,235.600006,191.449997
242,2023-11-21,3218.479980,336.980011,241.199997,190.639999
243,2023-11-22,3230.649902,341.489990,234.210007,191.309998


In [66]:
df_cash = pd.DataFrame([["2022-12-01", 50_000],
["2023-01-01", 50_000],
["2023-02-01", 50_000],
["2023-03-01", 50_000],
["2023-04-01", 50_000],
["2023-05-01", 50_000],
["2023-06-01", 50_000],
["2023-07-01", 50_000],
["2023-08-01", 50_000],
["2023-09-01", 50_000],
["2023-10-01", 50_000],
["2023-11-01", 50_000],
], columns = ["Date", "cashflow"])

In [4]:
def static_rebalance(current_assets: list,
                     current_prices: list,
                     proportions: list,
                     cash_flow: float) -> list:
    
    """This function takes lists of current assets (number of each), 
    their current prices, needed proportions of values (estimated through theory),
    and cash flow (deposit or withdraw).
    It rerurns the list of rebalanced numbers of each asset"""
    
    # len of all lists should be the same
    if (len(current_assets) != len(current_prices)) or (len(current_assets) != len(proportions)):
        raise ValueError('All lists should have same size')
    
    # calculating total value of our current shares
    total_value = 0
    for i in range(len(current_assets)):
        total_value += current_assets[i] * current_prices[i]
    
    # add deposit/withdraw
    total_value += cash_flow
    if total_value < 0:
        raise ValueError('Too big withdraw')
    
    # calculating new numbers
    rebalanced_assets = []
    for i in range(len(proportions)):
        rebalanced_assets.append(total_value * proportions[i] / current_prices[i])
        
    return rebalanced_assets



In [10]:
assets = [0, 2, 3]
prices = [10, 20, 10]
proportions = [0.3, 0.3, 0.4]
cash = 100

In [11]:
static_rebalance(assets, prices, proportions, cash)

[5.1, 2.55, 6.8]

In [ ]:
for i in range(len(df_cash)):
    date = df_cash["Date"][i]

In [69]:
df_cash["Date"][0]

'2022-12-01'

In [111]:
display(df_cash, df_final)

,Date,cashflow
0,2022-12-01,50000
1,2023-01-01,50000
2,2023-02-01,50000
3,2023-03-01,50000
4,2023-04-01,50000
5,2023-05-01,50000
6,2023-06-01,50000
7,2023-07-01,50000
8,2023-08-01,50000
9,2023-09-01,50000


,Date,IMOEX.ME,META,TSLA,AAPL
0,2022-11-28,2177.449951,108.779999,182.919998,143.418365
1,2022-11-29,2185.320068,109.459999,180.830002,140.385315
2,2022-11-30,2174.530029,118.099998,194.699997,147.207169
3,2022-12-01,2187.290039,120.440002,194.699997,147.485626
4,2022-12-02,2180.439941,123.489998,194.860001,146.988403
...,...,...,...,...,...
240,2023-11-17,3206.340088,335.040009,234.300003,189.690002
241,2023-11-20,3207.360107,339.970001,235.600006,191.449997
242,2023-11-21,3218.479980,336.980011,241.199997,190.639999
243,2023-11-22,3230.649902,341.489990,234.210007,191.309998


In [109]:
start_assets = {"IMOEX.ME": 0, "META": 0, "TSLA": 0, "AAPL": 0}
assets_number = list(start_assets.values())
assets_tickers = list(start_assets.keys())
needed_proportions = [0.2, 0.4, 0.1, 0.3]

for i in range(len(df_cash)):
    date = df_cash["Date"][i]
    
    #get cashflow for this date
    cashflow = df_cash.loc[df_cash["Date"] == date]["cashflow"].values[0]
    
    #get current assets prices
    c_prices = []
    for one_ticker in assets_tickers:
        j = 0
        one_price = df_final.loc[df_final["Date"] == pd.to_datetime(date)- pd.DateOffset(day=j)][one_ticker].values
        while len(one_price) == 0:
            one_price = df_final.loc[df_final["Date"] == pd.to_datetime(date)- pd.DateOffset(day=j)][one_ticker].values
            j += 1
        one_price = one_price[0]
        
        c_prices.append(one_price)
        
    assets_number = static_rebalance(assets_number, c_prices, needed_proportions, cashflow)
    

/var/folders/md/7rmqqy_j78vf1wdr47tnbjw00000gn/T/ipykernel_91836/335930958.py:16: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  one_price = df_final.loc[df_final["Date"] == pd.to_datetime(date)- pd.DateOffset(day=j)][one_ticker].values
/var/folders/md/7rmqqy_j78vf1wdr47tnbjw00000gn/T/ipykernel_91836/335930958.py:18: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  one_price = df_final.loc[df_final["Date"] == pd.to_datetime(date)- pd.DateOffset(day=j)][one_ticker].values


In [110]:
assets_number

[47.248275040779085, 971.6372412465947, 368.3325321132665, 1308.0025642353899]

In [115]:
tv = 0
for i in range(len(c_prices)):
    tv += c_prices[i]*assets_number[i]

In [116]:
tv

757512.6990328842

In [117]:
12*50_000

600000

In [ ]:
\